In [1]:
'''
Scrapy shell Xpath testing command:


scrapy shell -s USER_AGENT="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36" "https://www.chartable.com/charts/spotify/united-states-of-america-top-podcasts"  

"https://chartable.com/podcasts/the-joe-rogan-experience" 
'''
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
from datetime import date
import pandas as pd
import numpy as np

In [2]:
#Add User agent to scrape the site
from selenium.webdriver.chrome.options import Options
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36" 
)

In [3]:
from configparser import ConfigParser
cfg = ConfigParser(interpolation=None) 
cfg.read('conf.ini')
login_conf = cfg['chartable']
user_email = login_conf['email'] 
passwd = login_conf['passwd']

In [5]:
# Test Login to Chartable


driver = webdriver.Chrome(options=opts)
driver.get('https://www.chartable.com/sign_in')

email = driver.find_element_by_id('Email')

password = driver.find_element_by_id('Password')

email.send_keys(user_email)
password.send_keys(passwd)

driver.find_element_by_name('commit').click()
time.sleep(5)
driver.close()


# Scraping top 50 chart 

In [4]:
driver = webdriver.Chrome(options=opts)

# Start .csv file for top 50 chart
csv_chart_file = open('spotify_top50podcasts.csv', 'w', encoding='utf-8', newline='')
chart_writer = csv.writer(csv_chart_file)

#Open Google Chrome bot
start_page = 'https://www.chartable.com/charts/spotify/united-states-of-america-top-podcasts'
driver.get(start_page)

#Get ranks of all on the page
ranks_elems = driver.find_elements_by_xpath('//div[@class = "b header-font f2 tc"]')
ranks = [int(rank.text) for rank in ranks_elems]
rows = driver.find_elements_by_xpath('//td[@class = "pv2 ph1"]')
for row in rows:
    podcast_dict = {}
    try:
        url = row.find_element_by_xpath('./div[@class = "title f3"]/a').get_attribute('href')
    except:
        ranks.pop(0)
        continue
    if row.text.find('\n') ==-1:
        name = row.text
        network = 'Unaffiliated'
    else:
        network, name= row.text.split('\n')
    
    podcast_dict['rank']=ranks.pop(0)
    podcast_dict['name'] = name
    podcast_dict['network'] = network
    podcast_dict['date_scraped'] = date.today().strftime('%Y-%m-%d')
    podcast_dict['url'] = url
    chart_writer.writerow(podcast_dict.values())
csv_chart_file.close()
driver.close()

# Scraping episode info

In [22]:
driver = webdriver.Chrome(options=opts)
driver.get('https://www.chartable.com/sign_in')

email = driver.find_element_by_id('Email')

password = driver.find_element_by_id('Password')

email.send_keys(user_email)
password.send_keys(passwd)

driver.find_element_by_name('commit').click()
time.sleep(5)
chart50 = pd.read_csv('spotify_top50podcasts.csv', names = ['rank','name','network','date_scraped','url'])    


csv_podcast_file = open('podcast_data.csv', 'w', encoding='utf-8', newline='') 
podcast_writer = csv.writer(csv_podcast_file)


    
    
#Transfer to the podcast page
for url in chart50['url']:
    driver.get(url)

    try:
        find_genre = driver.find_elements_by_xpath('//div[@class = "links bg-white pa3 br2 b--near-white ba f6"]//div/a[contains(@href,"genre")]')
        try:
            genre = find_genre.text
        except:
            genre = [i.text for i in find_genre]
    except:
        genre = 'Unknown'

    try:
        stars_ratings = driver.find_element_by_xpath('//div[@class = "gray"]').text
        stars, ratings = stars_ratings.split(' stars from ')
        try:
            ratings = int(ratings.replace(',','').replace('ratings',''))
        except:
            ratings = int(ratings.replace('ratings',''))
    except:
        stars = None
        ratings = None

    podcast_dict = {}
    
    podcast_dict['genre'] = genre
    podcast_dict['stars'] = stars
    podcast_dict['ratings'] = ratings
    podcast_dict['url'] = url



    episodes_url = driver.find_element_by_xpath('//div[@class = "link mb2"]/a').get_attribute('href')
    time.sleep(4)
    driver.get(episodes_url)

    episodes = driver.find_elements_by_xpath('//div[@class = "mb4"]')
    for episode in episodes:
        episode_info = episode.find_elements_by_xpath('.//div')
        episode_info = [i.text for i in episode_info]
        if episode_info:
            episode_date = episode_info.pop()
        else:
            continue
        podcast_dict['episode_date'] = episode_date.replace('Published ','')
        
        podcast_writer.writerow(podcast_dict.values())   
    
    time.sleep(4)

    
csv_podcast_file.close()
driver.close()

In [13]:
csv_podcast_file.close()

In [11]:
driver.close()

## Run line below if error leaves csv/driver open

In [39]:
try:
    csv_file.close()
    driver.close()
except:
    driver.close()

In [ ]:
df_genre = pd.merge(df_genre.drop('genres', axis=1), genre, how='right', left_index=True, right_on='index')
df_genre.head()

### Working on getting names

In [ ]:
wait_ranks =WebDriverWait(driver, 10)
podcast_dict = {}
ranks = wait_ranks.until(EC.presence_of_all_elements_located((By.XPATH,'//div[@class = "b header-font f2 tc"]')))
for rank in ranks:
    num_rank = review.find_element_by_xpath('//div[@class = "b header-font f2 tc"]').text
    podcast_dict['rank'] = num_rank

In [41]:
podcast = pd.read_csv('podcast_data.csv', names = ['genres', 'stars', 'ratings','url','episode_date'])
podcast.head()

,genres,stars,ratings,url,episode_date


In [20]:
podcast = pd.read_csv('podcast_data.csv', names = ['genres', 'stars', 'ratings','url','episode_date'])
podcast.stars.value_counts()

4.7    100
4.6     80
4.5     50
3.9     25
4.9     25
4.1     25
4.4     25
4.3     25
Name: stars, dtype: int64

## Data reading/cleaning process

In [37]:
podcast = pd.read_csv('podcast_data.csv', names = ['genres', 'stars', 'ratings','url','episode_date'])
spotify_data= chart50.merge(podcast, on = 'url')
spotify_data.drop('url',axis = 1, inplace = True)
spotify_data.to_csv('podcast_cleaning.csv', index = False)

In [41]:
spotify = pd.read_csv('podcast_cleaning.csv')
spotify['episode_date'] = pd.to_datetime(spotify['episode_date'])
spotify['episode_day_of_week'] = spotify['episode_date'].dt.day_name()


def clean_genres(obj):
    if obj.find('[') == -1:
        return obj
    else:
        return obj[2:-2].replace("'",'').split(', ')
    
def list_unpacking(df,colname):
    index = 0
    list_ = []
    for item in df[colname]:
        list_.extend(map(lambda x: [index,x],item))
        index +=1
    new_df = pd.DataFrame(list_, columns = ['index', colname[:-1]])
    df = df.merge(new_df,left_index = True, right_on = 'index')
    df.drop(['genres','index'],axis = 1, inplace = True)
    return df

spotify.genres = spotify['genres'].apply(lambda s: clean_genres(s))

spotify['my_index'] = spotify.stars**2/np.log(spotify.ratings)

                                            
genre_analysis = spotify.copy()
genre_analysis= list_unpacking(spotify,'genres')
import numpy as np


#then join from index
genre_analysis.loc[genre_analysis.genre == 'Football']

,rank,name,network,date_scraped,stars,ratings,episode_date,episode_day_of_week,my_index,genre
656,16,Pardon My Take,Barstool Sports,2021-02-12,4.9,75069.0,2021-02-12,Friday,2.138754,Football
658,16,Pardon My Take,Barstool Sports,2021-02-12,4.9,75069.0,2021-02-10,Wednesday,2.138754,Football
660,16,Pardon My Take,Barstool Sports,2021-02-12,4.9,75069.0,2021-02-08,Monday,2.138754,Football
662,16,Pardon My Take,Barstool Sports,2021-02-12,4.9,75069.0,2021-02-05,Friday,2.138754,Football
664,16,Pardon My Take,Barstool Sports,2021-02-12,4.9,75069.0,2021-02-03,Wednesday,2.138754,Football
666,16,Pardon My Take,Barstool Sports,2021-02-12,4.9,75069.0,2021-01-31,Sunday,2.138754,Football
668,16,Pardon My Take,Barstool Sports,2021-02-12,4.9,75069.0,2021-01-29,Friday,2.138754,Football
670,16,Pardon My Take,Barstool Sports,2021-02-12,4.9,75069.0,2021-01-27,Wednesday,2.138754,Football
672,16,Pardon My Take,Barstool Sports,2021-02-12,4.9,75069.0,2021-01-25,Monday,2.138754,Football
674,16,Pardon My Take,Barstool Sports,2021-02-12,4.9,75069.0,2021-01-22,Friday,2.138754,Football


In [63]:
genre_analysis['genre'].loc[genre_analysis.network == 'The Ringer & Bill Simmons']

1258    TV & Film
1259       Sports
1260    TV & Film
1261       Sports
1262    TV & Film
1263       Sports
1264    TV & Film
1265       Sports
1266    TV & Film
1267       Sports
1268    TV & Film
1269       Sports
1270    TV & Film
1271       Sports
1272    TV & Film
1273       Sports
1274    TV & Film
1275       Sports
1276    TV & Film
1277       Sports
1278    TV & Film
1279       Sports
1280    TV & Film
1281       Sports
1282    TV & Film
1283       Sports
1284    TV & Film
1285       Sports
1286    TV & Film
1287       Sports
1288    TV & Film
1289       Sports
1290    TV & Film
1291       Sports
1292    TV & Film
1293       Sports
1294    TV & Film
1295       Sports
1296    TV & Film
1297       Sports
1298    TV & Film
1299       Sports
1300    TV & Film
1301       Sports
1302    TV & Film
1303       Sports
1304    TV & Film
1305       Sports
1306    TV & Film
1307       Sports
Name: genre, dtype: object

In [7]:
chart50.sample(10)

,rank,name,network,date_scraped,url
6,7,Call Her Daddy,Unaffiliated,2021-02-12,https://chartable.com/podcasts/call-her-daddy-...
29,30,The Bill Simmons Podcast,The Ringer & Bill Simmons,2021-02-12,https://chartable.com/podcasts/the-bill-simmon...
42,44,Taurus Today,Parcast Network,2021-02-12,https://chartable.com/podcasts/taurus-today
31,32,Conan O’Brien Needs A Friend,Team Coco & Earwolf,2021-02-12,https://chartable.com/podcasts/conan-obrien-ne...
41,43,Virgo Today,Parcast Network,2021-02-12,https://chartable.com/podcasts/virgo-today
8,9,Unlocking Us with Brené Brown,Parcast Network,2021-02-12,https://chartable.com/podcasts/unlocking-us-wi...
37,38,Tmsoft's White Noise Sleep Sounds,TMSOFT,2021-02-12,https://chartable.com/podcasts/tmsofts-white-n...
47,49,Aries Today,Parcast Network,2021-02-12,https://chartable.com/podcasts/aries-today
28,29,Conspiracy Theories,Parcast Network,2021-02-12,https://chartable.com/podcasts/conspiracy-theo...
38,39,Leo Today,Parcast Network,2021-02-12,https://chartable.com/podcasts/leo-today


In [ ]:
'''
scrape the entire chart

save to csv

load csv

if url:
go to his 

save that 



#make sure genre unpacking is for genre analysis only
'''

In [7]:
chart50 = pd.read_csv('spotify_top50podcasts.csv', names = ['rank','name','network','date_scraped','url'])
chart50.sample()

,rank,name,network,date_scraped,url
40,42,Anything Goes with Emma Chamberlain,Emma Chamberlain and Ramble,2021-02-12,https://chartable.com/podcasts/stupid-genius-w...


In [9]:
driver.close()

In [54]:
y = [i**2 for i in x].pop()

In [30]:
spotify_data.sample(20)

,rank,name,network,date_scraped,url,genres,stars,ratings,episode_date
367,16,Pardon My Take,Barstool Sports,2021-02-12,https://chartable.com/podcasts/pardon-my-take,"['Sports', 'Football']",4.9,75069.0,2021-01-15
1142,49,Aries Today,Parcast Network,2021-02-12,https://chartable.com/podcasts/aries-today,"['Society & Culture', 'Health & Fitness']",4.4,75.0,2021-02-01
846,36,Cancer Today,Parcast Network,2021-02-12,https://chartable.com/podcasts/cancer-today,"['Society & Culture', 'Health & Fitness']",4.7,100.0,2021-01-28
865,37,Planet Money,NPR,2021-02-12,https://chartable.com/podcasts/planet-money,"['Business', 'News']",4.6,32640.0,2021-01-11
462,20,Serial Killers,Parcast Network,2021-02-12,https://chartable.com/podcasts/serial-killers-...,"['History', 'True Crime']",4.7,34130.0,2021-01-21
179,8,My Favorite Murder with Karen Kilgariff and Ge...,Unaffiliated,2021-02-12,https://chartable.com/podcasts/my-favorite-mur...,[],NaN,NaN,2020-08-10
538,23,Supernatural with Ashley Flowers,Parcast Network,2021-02-12,https://chartable.com/podcasts/supernatural-wi...,"['Society & Culture', 'True Crime']",4.7,18345.0,2020-12-16
1089,47,The Tiny Meat Gang Podcast,Tiny Meat Gang,2021-02-12,https://chartable.com/podcasts/the-tiny-meat-g...,['Comedy'],4.9,36891.0,2020-12-18
297,13,Armchair Expert with Dax Shepard,Armchair Umbrella,2021-02-12,https://chartable.com/podcasts/armchair-expert...,"['Comedy', 'TV & Film']",4.7,72750.0,2021-01-14
311,14,Deep Sleep Sounds,Slumber Group,2021-02-12,https://chartable.com/podcasts/deep-sleep-sounds,['Kids & Family'],4.4,487.0,2021-01-04


In [15]:
import re
x = ['no', 'no','published 9/15/2020']
re.search('published (\d+)\/(\d+)\/(\d{4})',x[2])

<re.Match object; span=(0, 19), match='published 9/15/2020'>

In [18]:
episode_info

['#1571 - Emily Harrington',
 "Rock climber and adventurer Emily Harrington is a five-time US National Champion in Sport Climbing. She has scaled some of the world's most formidable mountains, including Everest, Ama Dablam, and Cho Oyu, and is the first woman to free climb El Capitan via Golden Gate in under 24 hours.",
 'Published 11/27/20']